In [ ]:
import numpy as np
import nrrd
from os.path import *
from pylab import *
from matplotlib import patches
from JSONread import *
from density_function import *
import xlrd
import json

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "output/"

In [ ]:
SMALL_SIZE = 24
BIGGER_SIZE = 30

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
# Loading annotations
annotation, h = nrrd.read(join(DATA_FOLDER, "annotations.nrrd"))
jsontextfile = open(join(DATA_FOLDER, "brain_regions.json"), "r")
jsoncontent = json.loads(jsontextfile.read())
search_children(jsoncontent['msg'][0])

In [ ]:
num_neurons = json.loads(open(join(DATA_FOLDER, "neuron_counts.json"), "r").read())
# num_gad = json.loads(open(join(OUTPUT_FOLDER, "gad_counts.json"), "r").read())
# for k, v in num_neurons.items():
#     num_neurons[k] = v - num_gad[k] * 25**3/1.0e9

In [ ]:
wb = xlrd.open_workbook(join(DATA_FOLDER, "gaba_papers.xlsx")) 
hasInfo = np.zeros(len(region_keys), dtype=np.bool)
for i_sheet in range(2):
    sheet = wb.sheet_by_index(i_sheet)
    for i_region in range(sheet.nrows-1):
        name = sheet.cell_value(i_region+1, 0).replace("È", "e")
        if name in region_keys:
            hasInfo[region_keys.index(name)] = 1
        else:
            print(name)
for i, kkk in enumerate(region_keys):
    if not hasInfo[i]:
        parent_ = region_dictionary_to_id_parent[kkk]
        while parent_ != "":
            if hasInfo[region_keys.index(parent_)]:
                hasInfo[i] = True
                break
            parent_ = region_dictionary_to_id_parent[parent_]

In [ ]:
without_cereb = False
show_known = True 

if show_known:
    loc_hasInfo = hasInfo
    title_ = "Literature coverage for GAD67 densities in the mouse brain"
    filename = "literature_coverage"
else: 
    loc_hasInfo = None
    title_ = "Distribution of the neurons in the mouse brain"
#     title_ = "Distribution of the Excitatory neurons in the mouse brain"
    filename = "neu_counts" 
#     filename = "exc_counts"
if without_cereb:
    filename+="_without_cereb"

filename = join(OUTPUT_FOLDER, filename)
plot_circular_dens(num_neurons, title_, filename,
                   name2allname, region_keys, region_dictionary_to_id,
                   region_dictionary_to_id_parent, region_to_color,
                   hasInfo=loc_hasInfo, without_cereb=without_cereb)